Stacks and Queues

Think of caching

Stacks -> last in, first out  
top (same as peek), push (or emplace), pop(removes **not** retrieves)  
when stack is .empty() -> top and pop throw exceptions   

Queues -> first in, first out  

In [1]:
template <typename T>
struct Node
{
    T data;
    std::shared_ptr<Node<T>> next;
};

In [2]:
#include <stack>
#include <iostream>

//time and space is O(n)
void PrintLinkedListinReverse(std::shared_ptr<Node<int>> head)
{
    std::stack<int> stack;
    
    while(head)
    {
        stack.push(head->data);
        head = head->next;
    }
    
    while(!stack.empty())
    {
        std::cout << stack.top() << std::endl;
        stack.pop();
    }
}

8.1 -> implement stack with max api  
max operation, in addition to push and pop  
max method should return the maximum value stored in the stack   

use of a cache

Btw review how below references work, must read through

In [7]:
double a;

In [8]:
//If no & ampersand, wouldn't work, wouldn't compile since not a modifiable lvalue
double& setValues() 
{
    return a;  
}

In [11]:
{
    a = 1;
    std::cout << "1. a: " << a << std::endl;

    setValues() = 2;
    std::cout << "2. a: " << a << std::endl;


    double x = setValues();
    x = 3;
    std::cout << "3. a: " << a << std::endl;

    double& y = setValues();
    y = 4;
    std::cout << "4. a: " << a << std::endl;

    x = setValues();
    x = 5;
    std::cout << "5. a: " << a << std::endl;

    y = setValues();
    y = 6;
    std::cout << "6. a: " << a << std::endl;

    y = 7;
    std::cout << "7. a: " << a << std::endl;
}

1. a: 1
2. a: 2
3. a: 2
4. a: 4
5. a: 4
6. a: 6
7. a: 7


Can improve best-case **space** by having a separate stack and storing the counts of a max with that max  
Ex: push in this order: 3, 3, 1, 4  
max_stack would store {3, 2}, {4, 1}  

8.2 -> Evaluate RPN expressions  

https://stackoverflow.com/questions/22514855/arrow-operator-in-function-heading  
-> int //is inferered return type in a lambda

https://stackoverflow.com/a/33239463  
Basically:  
count() -> if element is there, 0 or 1  
at() -> to access element knowing that it's there, ref to element  
\[\] -> if it doesn't exist will create it  
find() -> don't know if its there, returns an iterator to the element if it exists or an iterator to map::end() if it does not  
or instead of find() just use count(), then at()

In [ ]:
#include <stack>
#include <unordered_map>

int Evaluate(const std::string& expr)
{
    std::stack<int> stack;
    std::stringstream ss(expr);
    std::string token;

    const char delim = ',';

    std::unordered_map<char, std::function<int(int, int)>> map =
    {
        {'*', [](int x, int y) -> int { return x * y; }},
        {'-', [](int x, int y) -> int { return x - y; }},
        {'+', [](int x, int y) -> int { return x + y; }},
        {'/', [](int x, int y) -> int { return x / y; }},
    };

    while (std::getline(ss, token, delim))
    {
        if (map.count(token[0]))
        {
            const int y = stack.top(); stack.pop();
            const int x = stack.top(); stack.pop();
            stack.emplace(map.at(token[0])(x, y)); //always use .at() for these kind of cases
        }
        else {
            stack.emplace(std::stoi(token));
        }
    }

    return stack.top();
}


8.3 -> well formed-ness of brackest

In [ ]:
#include <stack>
#include <unordered_map>

bool isWellFormed(const std::string& s)
{
    std::stack<char> stack;
    std::unordered_map<char, char> map = 
    {
        {')', '('},
        {'}', '{'},
        {']', '['},

    };


    for (char c : s)
    {
        if(map.count(c))
        {
            if(!stack.empty() && map.at(c) == stack.top())
                stack.pop();
            else
                return false;
        }
        else
        {
            stack.emplace(c);
        }
    }

    return !stack.size();
}

In [ ]:
#include <stack>
#include <string>
#include <unordered_map>

bool IsWellFormed(const std::string& s)
{
    std::stack<char> stack;
    const std::unordered_map<char, char> map
    {
        {'(', ')'},
        {'[', ']'},
        {'{', '}'}
    };

    for (char c : s)
    {
        if (!stack.empty() && map.at(stack.top()) == c)
        {
            stack.pop();
        }
        else if (map.count(c)) {
            stack.emplace(c);
        } else {
            return false;
        }
    }
    return stack.empty();
}

Queue: insertion at back, pop at front    
dequeue -> from head  
enque -> at tail  
c++ -> front() (just retrieves), back() (just retrieves), push(42), emplace(42), void pop()  
when empty, front(), back, and pop() throw exceptions  

front, back, push, pop    
emplace == push  

Deque:  
Insertion: push_front()/emplace_front(), inject -> push_back()/emplace_back()  
Deletion: pop_front(), eject -> pop_back()  
front(), back() -> peek  

front, back, push_front, push_back, pop_front, pop_back  
emplace == push  

Btw: https://stackoverflow.com/a/8452900  
https://stackoverflow.com/a/7593152  
std::begin(lol) -> allows for more flexibility, works for arrays  
lol.begin() -> not for arrays  


This works since it increases the original pointer value, since & it's not a copy  
This is a generic to find the end of an array  

```c++
template<typename T, size_t N> T* end(T (&a)[N]) { return a + N; }
```

In [7]:
#include <list>

class Queue
{
    private:
        std::list<int> data;
    public:
        void Enqueue(int x)
        { 
            data.emplace_back(x); 
        }     
        
        //deeeeequeue not the dequeue
        int Dequeue()
        {
            if(data.empty())
            {
                throw length_error("empty queue");
            }
            const int val = data.front();
            data.pop_front();
            
            return val;
        }
    
        int Max() const
        {
            if(data.empty())
            {
                throw length_error("cannot perform Max() on empty queue");
            }
            
            for(std::list<int>::iterator i = data.begin(); i != data.end(); i++)
                std::cout << *i << std::endl;
            //don't have to use this function, but can use above function instead
            return *std::max_element(data.begin(), data.end());
        }
};

input_line_17:1:7: error: redefinition of 'Queue'
class Queue
      ^
input_line_13:1:7: note: previous definition is here
class Queue
      ^


Interpreter Error: 

8.6 -> Return a vector of vectors, each vector has all the nodes on the same level

In [8]:
template <typename T>
struct BinaryTreeNode
{
    T data;
    std::unique_ptr<BinaryTreeNode*> left, right;
};

std::unique_ptr<T> a; a.get() -> returns raw pointer

Problem requests:  
1. The way the book says bruteforce, inorder traversal  
2. Breadth first search, one queue with nodes and one vector with struct of BinaryTreeNode and key value 

Below code is normal breadth first search  
Problem is that each "level" will only have two nodes max in each  
Needs to consider all nodes on the same level index  

In [ ]:
#include <vector>
#include <queue>

std::vector<std::vector<int>> BinaryTreeDepthOrder(const std::shared_ptr<BinaryTreeNode>& tree)
{
    std::vector<std::vector<int>> res;
    
    if(!tree.get())
        return nodes;
    
    std::queue<BinaryTreeNode> q;    
    std::shared_ptr<Node> i = tree;
    q.push(i);
    res.push({i});
    
    while(!q.empty())
    {
        std::shared_ptr<BinaryTreeNode> e = q.front(); q.pop();
        
        std::vector<int> level;
        if(e->left)
        {
            level.push_back(e->left->data);
            q.push(e->left);
        }
        
        if(e->right)
        {
            level.push_back(e->right->data);
            q.push(e->right);
        }
        
        res.push_back(level);
    }
    
    return res;
}

Problem Request: reproduce this - https://www.geeksforgeeks.org/tree-traversals-inorder-preorder-and-postorder/    
preorder, postorder, inorder traversals  

In [ ]:
#include <vector>
#include <queue>

std::vector<std::vector<int>> BinaryTreeDepthOrder(const std::unique_ptr<BinaryTreeNode<int>>& tree)
{
    std::vector<std::vector<int>> result;
    if (!tree.get())
        return result;

    //curr nodes can be for nodes at a level, 1st level 1 node, 2nd level 2 nodes, etc...
    std::queue<BinaryTreeNode<int>*> curr_nodes;
    curr_nodes.emplace(tree.get());

    while (!curr_nodes.empty()) //for each level
    {
        std::vector<int> level;
        std::queue<BinaryTreeNode<int>*> next_nodes;

        while (!curr_nodes.empty()) //for each node in each level
        {
            BinaryTreeNode<int>* curr = curr_nodes.front(); curr_nodes.pop();
            level.emplace_back(curr->data);

            if (curr->left)
                next_nodes.emplace(curr->left.get()); //remember, the BinaryTreeNode struct contains a smart pointer
                //therefore must do .get() to get raw pointer to match type of queue

            if (curr->right)
                next_nodes.emplace(curr->right.get());
        }

        result.emplace_back(level);
        curr_nodes = next_nodes;
    }

    return result;
}

8.7 -> Implement a ciruclar queue

Explicit is used to avoid ambiguity in overloaded methods: void foo(int); or void foo(char* p);  
https://stackoverflow.com/questions/5499085/c-overload-constructor-with-int-and-char  
https://www.geeksforgeeks.org/function-overloading-c/  

Explicit is also used to avoid implicit conversions, FooObject a = 32; //when constructor is FooObject(int sdfs)
https://stackoverflow.com/questions/121162/what-does-the-explicit-keyword-mean  

Checkout the explicits in C++11 vector
http://www.cplusplus.com/reference/vector/vector/vector/

Side problem: given a list of numbers 1 to 100 in shuffled order, find missing number: add numbers together, subtract 100(100+1)/2 to get missing number

Problem Request: 
Side problem: gcd -> https://www.khanacademy.org/computing/computer-science/cryptography/modarithmetic/a/the-euclidean-algorithm  
http://rosettacode.org/wiki/Greatest_common_divisor#C.2B.2B  

In [2]:
int gcd(int a, int b)
{
    return b == 0 ? a : gcd(b, a % b);
}

https://stackoverflow.com/a/23321270  

Side problem: rotate array  
n -> number of elements  
k -> number of movements  
if n/k is evenly divisble, then there are k sets/cycles   
do this visually  
otherwise n sets/cycles

gcd(n,k) 


In [ ]:
void rotate_c(std::vector<int>& vec, int k)
{
    int n = vec.size();
    int cycles = gcd(n, k);

    for (int i = 0; i < cycles; i++)
    {
        int prev = i;
        int tmp = vec[i];
        int curr = (prev + k) % n;

        while (curr != i)
        {
            vec[prev] = vec[curr];
            prev = curr;
            curr = (prev + k) % n;
        }
        vec[prev] = tmp;
    }
}

void reverse(std::vector<int>& vec, int begin, int end)
{
    for (int i = begin; i <= (end + begin) / 2; i++)
    {
        int a = vec[i];
        vec[i] = vec[end - (i - begin)];
        vec[end - (i - begin)] = a;
    }
}

void rotate_r(std::vector<int>& vec, int k)
{
    reverse(vec, 0, vec.size() - 1);
    reverse(vec, 0, vec.size() - 1 - k);
    reverse(vec, vec.size() - k, vec.size() - 1);
}

In [ ]:
class Queue
{
    private:
        const int kScaleFactor = 2;
        int head = 0, tail = 0;
        int num_queued_elements = 0;
        std::vector<int> vec;
        
    public:
        Queue(int capacity) : vec(capacity) {}
        void enqueue(int x)
        {
            if(num_queued_elements == vec.size())
            {
                rotate_c(vec, head);
                vals.resize(num_queued_elements * kScaleFactor);
                head = 0, tail = num_queued_elements;
            }
            vec[tail] = x;
            tail = (tail + 1) % vec.size(), num_queued_elements++;
        }
        
        int dequeue()
        {
            if(!num_queued_elements)
                throw std::length_error("empty queue");
            
            num_queued_elements--;
            int tmp = vec[head];
            head = (head + 1) % vec.size();
            return tmp;
        }
    
        int size() const
        {
            return vec.size();
        }
};

8.8 Queue using stacks

In [ ]:
#include <stack>

class Queue
{
    private:
        std::stack<int> enqueue, dequeue;
    public:
        void enqueue(int x)
        {
            enqueue.push(x);
        }
    
        void dequeue(int x)
        {
            if(dequeu.empty())
            {
                while(!enqueue.empty())
                {
                    dequeue.push(enqueue.top())
                    enqueue.pop();
                }
                
                int tmp = dequeue.top();
                dequeue.pop();
                return tmp;
            }
        }
}